In [78]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
from random import random

In [86]:
df

,値段,株数,金額,時間
時刻,,,,
2024-05-14 09:00:00,1685,304300,512745500,09:00:00
2024-05-14 09:00:00,1685,500,842500,09:00:00
2024-05-14 09:00:00,1685,500,842500,09:00:00
2024-05-14 09:00:00,1685,6900,11626500,09:00:00
2024-05-14 09:00:00,1685,300,505500,09:00:00
...,...,...,...,...
2024-05-14 14:59:59,1697,200,339400,14:59:59
2024-05-14 14:59:59,1696,500,848000,14:59:59
2024-05-14 14:59:59,1696,300,508800,14:59:59


In [82]:
df = pd.read_csv(
    '~/Downloads/qr-7003-20240513.csv',
    index_col=3,
    parse_dates=True
    )[::-1]
df['時間'] = df.index.time

# 1分ごとにリサンプル
dfm = df['値段'].resample('1T').ohlc()
dfm['ticks'] = df['値段'].resample('1T').size()
dfm['volume'] = df['株数'].resample('1T').sum()
dfm

# 1秒ごとにリサンプル
dfs = df['値段'].resample('1S').ohlc()
dfs['ticks'] = df['値段'].resample('1S').size()
dfs['volume'] = df['株数'].resample('1S').sum()
dfs[:'2024-05-14 09:05']


/var/folders/_g/xsh34k3n5s70gm9jd9v92gvc0000gn/T/ipykernel_25572/1627483185.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,open,high,low,close,ticks,volume
時刻,,,,,,
2024-05-14 09:00:00,1685.0,1686.0,1683.0,1683.0,16,333200
2024-05-14 09:00:01,1684.0,1685.0,1681.0,1682.0,16,52000
2024-05-14 09:00:02,1683.0,1683.0,1674.0,1676.0,21,102900
2024-05-14 09:00:03,1674.0,1675.0,1672.0,1675.0,10,16000
2024-05-14 09:00:04,1676.0,1678.0,1674.0,1678.0,6,1500
...,...,...,...,...,...,...
2024-05-14 09:05:55,1658.0,1658.0,1658.0,1658.0,4,800
2024-05-14 09:05:56,1658.0,1658.0,1658.0,1658.0,1,100
2024-05-14 09:05:57,1658.0,1658.0,1658.0,1658.0,1,100


In [83]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dfs = dfs[:'2024-05-14 09:30']


# ローソク足チャートの作成
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Candlestick(x=dfs.index,
                             open=dfs['open'],
                             high=dfs['high'],
                             low=dfs['low'],
                             close=dfs['close'],
                             name='OHLC'),
              row=1, col=1)

# 出来高のヒストグラムの作成
fig.add_trace(go.Bar(x=dfs.index, y=dfs['volume'], name='Volume'),
              row=2, col=1)

# tick数のヒストグラムの作成
fig.add_trace(go.Bar(x=dfs.index, y=dfs['ticks'], name='Ticks'),
              row=3, col=1)

# レイアウトの設定
fig.update_layout(height=800, width=1000, title_text="OHLC with Volume and Ticks")

# チャートの表示
fig.show()


In [73]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import re

# データの読み込み
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv")

# 日付の整形
for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

# サブプロットの作成
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}], [{"type": "scatter"}], [{"type": "scatter"}]]
)

# チャートの追加
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Mining-revenue-USD"],
        mode="lines",
        name="mining revenue"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Hash-rate"],
        mode="lines",
        name="hash-rate-TH/s"
    ),
    row=2, col=1
)

# テーブルの追加
fig.add_trace(
    go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)", "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD", "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
    ),
    row=1, col=1
)

# レイアウトの設定
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Bitcoin mining stats for 180 days",
)

# 表示
fig.show()
